In [ ]:
!pip install pymongo
!pip install dnspython
!pip install certifi

In [1]:
%env MONGO_ATLAS_HOST=
%env MONGO_ATLAS_USERNAME=
%env MONGO_ATLAS_PASSWORD=

env: MONGO_ATLAS_HOST=iotaimpactcluster.vqlak.mongodb.net
env: MONGO_ATLAS_USERNAME=development
env: MONGO_ATLAS_PASSWORD=development


In [2]:
import os
import certifi
from pymongo import MongoClient
from bson.json_util import dumps, ObjectId
import gridfs

In [3]:
def get_db_handle(**kwargs):
    ''' Database connector
    Optional Arguments:
        host, username, password, db_name
    Return:
        client: Mongo Client
        db_handle: client[db_name]
    '''

    host = kwargs.get('host', os.environ["MONGO_ATLAS_HOST"])
    username = kwargs.get('username', os.environ["MONGO_ATLAS_USERNAME"])
    password = kwargs.get('password', os.environ["MONGO_ATLAS_PASSWORD"])
    db_name = kwargs.get('db_name', 'iotaimpact')
    try:
        url = f"mongodb+srv://{username}:{password}@{host}/{db_name}?retryWrites=true&w=majority"
        client = MongoClient(url, tlsCAFile=certifi.where())

        db_handle = client[db_name]
        return db_handle, client

    except Exception as e:
        print(f'Something went wrong: {e} \nTrying to re-connect to Mongodb...')
        return get_db_handle()

def insert(**kwargs):
    ''' General Purpose INSERT
    Arguments: 
        db_name (optional): target database (default: iotaimpact)
        collection: target collection
        document(s): Object or List of Objects to be stored
    Return: 
        ID(s) of the inserted object(s)
    '''
    db_name = kwargs.get('db_name', 'iotaimpact')
    db, _ = get_db_handle(db_name=db_name)

    collection = kwargs.get('collection', 'undefined_collection')
    document = kwargs.get('document', None)
    documents = kwargs.get('documents', None)
    
    # insert_one
    if document:
        return str(db[collection].insert_one(document).inserted_id)
    # insert_many
    elif documents:
        return str(db[collection].insert_many(documents))

def update(**kwargs):
    db_name = kwargs.get('db_name', 'iotaimpact')
    db, _ = get_db_handle(db_name=db_name)

    query_filter = kwargs.get('query_filter', None)
    collection = kwargs.get('collection', None)
    document = kwargs.get('document', None)

    if collection is None or query_filter is None or document is None:
        raise Exception('Specify collection, query_filter and document')
    
    return str(db[collection].update_one(query_filter, {'$set': document}))

def find(**kwargs):
    ''' General Purpose FIND 
    Arguments: 
        db_name (optional): target database
        collection: target collection
        query_filter: mongodb find query_filter
    Return: 
        list of documents found
    '''
    db_name = kwargs.get('db_name', 'iotaimpact')
    db, _ = get_db_handle(db_name=db_name)

    collection = kwargs.get('collection', None)
    if collection is None:
        raise Exception('Specify the collection')

    id = kwargs.get('id', None) 
    query_filter = kwargs.get('query_filter', None) # query_filter = { user_id: user_id }
    search_result = None
  
    if id:
        search_result = dumps(db[collection].find_one({"_id": ObjectId(id)}))

    elif query_filter: 
        search_result = dumps(db[collection].find(query_filter))

    return search_result

# GridFS Methods
def insert_large_document(binary_document, **kwargs):
    # Document validation
    if not binary_document:
        raise Exception('Specify the document to save')

    # Get the database client
    db_name = kwargs.get('db_name', 'iotaimpact')
    db, _ = get_db_handle(db_name=db_name)

    # Mongodb - GridFS storage
    return put(db, binary_document, **kwargs)

In [4]:
# GridFS Put Method
def put(db, binary_document, **kwargs):
    ''' It is the equivalent of creating a new file and writing on it '''
    fs = gridfs.GridFS(db)
    # return fs.put(binary_document, **kwargs)

    fs.put(binary_document)
    return

In [5]:
string_document = 'Hola Mundo String'
binary_document = b'Hola Mundo Bytes'
filename='test'

In [6]:
db, client = get_db_handle(db_name='iotaimpact')

In [7]:
client.admin.command('ping')

{'ok': 1}

In [8]:
db.list_collection_names()

['google_trends_analysis', 'surveys']

In [9]:
find(collection='surveys', id='61537d880c3e4caaa2b22c01')

'{"_id": {"$oid": "61537d880c3e4caaa2b22c01"}, "user_token": "186.113.136.245:2021-09-23", "survey": {"industry_f": "Cookie and Cracker Manufacturing ", "question2": "Products and services", "question9": "Pain points", "question18": ["Sizing of needs and pain points by customer profile"], "question31": [{}], "question33": "B2C distributor ", "question39": ["Overall needs and pain points for the individuals as they relate to all aspects of their lives"], "question40": [{}], "question42": [{}], "question66": [{}], "question69": [{}], "question71": [{}], "question90": [{}], "question99": "Food and beverage", "question108": [{}], "question111": [{}], "question113": [{}], "question114": [{}], "question116": [{}], "slider12": "50", "slider2": "50", "slider3": "50", "slider4": "50", "slider5": "50", "slider6": "50", "slider7": "50", "slider8": "50", "slider9": "50"}, "survey_answers": "Cookie and Cracker Manufacturing , Products and services, Pain points, Sizing of needs and pain points by cu

In [10]:
db, _ = get_db_handle(db_name='iotaimpact')
fs = gridfs.GridFS(db)

In [11]:
fs.put(binary_document, filename=filename)

ObjectId('615679954ea44edccce7a848')

In [ ]:
help(gridfs)